<a href="https://colab.research.google.com/github/YdoUcare-qm/LearningDeep/blob/main/PyTorch_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
import numpy as np
import math

In [101]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("souravsv/sample-csv-datasets","loan_prediction.csv")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/souravsv/sample-csv-datasets/versions/1/loan_prediction.csv


In [104]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [105]:
xy=pd.read_csv("/root/.cache/kagglehub/datasets/souravsv/sample-csv-datasets/versions/1/loan_prediction.csv")
cols_to_drop=['Loan_ID','Dependents']
xy=xy.drop(cols_to_drop,axis=1)

xy=xy.dropna(axis=0)

le=LabelEncoder()
for cat in ['Gender','Married','Education',
            'Self_Employed','Credit_History','Property_Area','Loan_Status']:
  xy[cat]=le.fit_transform(xy[cat])

xy=xy.values
ds=torch.tensor(xy)
X=ds[:,:10]
y=ds[:,10]

In [106]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

In [107]:
class LoanDataset(Dataset):
  def __init__(self,x,y):
    self.x=x
    self.y=y
  def __getitem__(self,index):
    return self.x[index],self.y[index]

  def __len__(self):
    return self.x.shape[0]

In [108]:
ds_train=LoanDataset(X_train,y_train)
ds_test=LoanDataset(X_test,y_test)
ds_train[0],len(ds_train),ds_test[0],len(ds_test)

((tensor([1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 5.5000e+03, 1.2600e+03,
          1.7000e+02, 3.6000e+02, 1.0000e+00, 0.0000e+00], dtype=torch.float64),
  tensor([1.], dtype=torch.float64)),
 392,
 (tensor([0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.9630e+03, 0.0000e+00,
          5.3000e+01, 3.6000e+02, 1.0000e+00, 1.0000e+00], dtype=torch.float64),
  tensor([1.], dtype=torch.float64)),
 98)

In [109]:
train_loader=DataLoader(ds_train,batch_size=4,shuffle=True)
test_loader=DataLoader(ds_test,batch_size=4,shuffle=True)

In [110]:
import torch.nn as nn
import torch

In [129]:
class LogisticRegression(nn.Module):
  def __init__(self,input_features):
    super(LogisticRegression,self).__init__()
    self.linear=nn.Linear(input_features,1,dtype=torch.float64)

  def forward(self,x):
    return torch.sigmoid(self.linear(x))


In [179]:
epochs=300
n_iter=math.ceil(len(ds_train)/4)
lr=0.01

In [180]:
model=LogisticRegression(10)
optimizer=torch.optim.SGD(model.parameters(),lr=lr)
loss=nn.BCELoss()

In [181]:
for epoch in range(epochs):
  eloss=0
  for i,(input,label) in enumerate(train_loader):
    y_pred=model(input)
    l=loss(y_pred,label)
    with torch.no_grad():
      eloss+=l
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
    # print(f'Iteration:{i}/{n_iter}')
  print(f'Epoch:{epoch+1}/{epochs} Loss:{(eloss/97):.6f}')

Epoch:1/300 Loss:35.040791
Epoch:2/300 Loss:32.216495
Epoch:3/300 Loss:32.216495
Epoch:4/300 Loss:32.216495
Epoch:5/300 Loss:32.216495
Epoch:6/300 Loss:32.216495
Epoch:7/300 Loss:32.216495
Epoch:8/300 Loss:32.216495
Epoch:9/300 Loss:32.216495
Epoch:10/300 Loss:32.216495
Epoch:11/300 Loss:32.216495
Epoch:12/300 Loss:32.216495
Epoch:13/300 Loss:32.216495
Epoch:14/300 Loss:32.216495
Epoch:15/300 Loss:32.216495
Epoch:16/300 Loss:32.216495
Epoch:17/300 Loss:32.216495
Epoch:18/300 Loss:32.216495
Epoch:19/300 Loss:32.216495
Epoch:20/300 Loss:32.216495
Epoch:21/300 Loss:32.216495
Epoch:22/300 Loss:32.216495
Epoch:23/300 Loss:32.216495
Epoch:24/300 Loss:32.216495
Epoch:25/300 Loss:32.216495
Epoch:26/300 Loss:32.216495
Epoch:27/300 Loss:32.216495
Epoch:28/300 Loss:32.216495
Epoch:29/300 Loss:32.216495
Epoch:30/300 Loss:32.216495
Epoch:31/300 Loss:32.216495
Epoch:32/300 Loss:32.216495
Epoch:33/300 Loss:32.216495
Epoch:34/300 Loss:32.216495
Epoch:35/300 Loss:32.216495
Epoch:36/300 Loss:32.216495
E

KeyboardInterrupt: 

In [115]:
from sklearn.metrics import accuracy_score,precision_score

In [174]:
y_pred=model(X_train)
y_pred=y_pred.detach().numpy()
y_pred=(y_pred>0.5).astype(np.int32)
acc=accuracy_score(y_pred,y_train)
prc=precision_score(y_pred,y_train)
print(f'Accuracy: {acc} Precision: {prc}')

Accuracy: 0.6811224489795918 Precision: 1.0


In [175]:
y_pred=model(X_test)
y_pred=y_pred.detach().numpy()
y_pred=(y_pred>0.5).astype(np.int32)
acc=accuracy_score(y_pred,y_test)
prc=precision_score(y_pred,y_test)
print(f'Accuracy: {acc} Precision: {prc}')

Accuracy: 0.7040816326530612 Precision: 1.0
